Version Date: April 30, 2023

# Molecular Dynamics (MD) Simulation of Gases

This notebook uses molecular dynamics (MD) to simulate the behavior of real noble gases.

#Objectives

- Gain proficiency in reading and modifying Python code in the Google Colab/Jupyter notebook environment
- Use molecular dynamics (MD) to simulate real noble gases with a Lennard-Jones potential
- Observe how gas identity, temperature, and number density affect deviations from ideality as measured by the compressibility factor (Z)
- Visualize MD trajectories using the NGLView widget

#Getting Started

First we need to install the necessary Python libraries. Run the code block below to load the libraries that we'll need for this simulation. (**Note:** this step may take a few minutes.)


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 19 13:56:08 2020
Last modified 20220706 by EST

@author: foleyj10
"""

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.spatial.distance as dist

### install and import MDAnalysis
#!pip install --upgrade MDAnalysis

#import MDAnalysis as MD

### install and import nglview
#!pip install nglview

#from google.colab import output
#output.enable_custom_widget_manager()

import nglview as ngl

Now we will execute the code that defines the MD simulation functions. Only run this code block once at the start of each session.

(**Note:** the default number of atoms in the simulation is 216, but this value can be changed if desired by editing the `num_atoms` variable in the first line of code.)

In [2]:
### simulation code

### define number of atoms (default N is 216)
num_atoms = 216

class noble_gas:
  def __init__(self, atom_name, initial_temp, density):
    ''' set some default values for the number
    of particles, the temperature, the volume, and the gas 
    identity, which will then determine the L-J parameters and
    conversion factors from natural to SI units! '''
    ### Avagadros number
    self.NA = 6.022140857e23
    ### Boltzman's constant in SI units
    self.kb_SI = 1.38064852e-23
    ### default N is 216
    self.N = num_atoms
    ### default atom is Ar
    self.atom = atom_name
    ### default length of container is 6.1822e+01 natural units... this
    ### gives about 40 moles / m^3 for Argon
    self.L_nu = 6.1822e1
    ### default temperature in Kelvin
    self.T_SI = initial_temp
    ### conversion factors depending on atom type (need to complete for all nobel gas atom types)
    if self.atom=='Ar':
      self.vol_fac = 3.7949992920124995e-29
      self.press_fac = 51695201.06691862
      self.temp_fac = 142.0950000000000
      self.time_fac = 2.09618e-12
    elif self.atom=='He':
      self.vol_fac = 1.8399744000000005e-29
      self.press_fac = 8152287.336171632
      self.temp_fac = 10.864459551225972
      self.time_fac = 1.7572698825166272e-12
    elif self.atom=='Ne':
      self.vol_fac = 2.0570823999999997e-29
      self.press_fac = 27223022.27659913
      self.temp_fac = 40.560648991243625
      self.time_fac = 2.1192341945685407e-12
    elif self.atom=='Kr':
      self.vol_fac = 4.5882712000000004e-29
      self.press_fac = 59935428.40275003
      self.temp_fac = 199.1817584391428
      self.time_fac = 8.051563913585078e-13
    elif self.atom=='Xe':
      self.vol_fac =  5.4872e-29
      self.press_fac = 70527773.72794868
      self.temp_fac = 280.30305642163006
      self.time_fac =  9.018957925790732e-13
    else:
      self.vol_fac = 3.7949992920124995e-29
      self.press_fac = 51695201.06691862
      self.temp_fac = 142.0950000000000
      self.time_fac = 2.09618e-12


    ### get important attributes in natural units for the simulation

    ### default time-step  s.t. in SI it is 10 f.s.
    self.dt = 1e-14 / self.time_fac

    ### temperature in natural units
    self.T = self.T_SI / self.temp_fac 

    ### volume in SI units
    self.vol_si = self.N/(density * self.NA)
    ### volume in natural units
    self.vol = self.vol_si / self.vol_fac
    ### length in natural units
    self.L_nu = self.vol ** (1./3)

    ### mass in natural units
    self.mass = 1.

    ### L-J sigma in natural units
    self.sigma = 1.
    ### L-J epsilon in natural units
    self.epsilon = 1. 

    ### quantities that need to be computed after initialization
    ### kinetic energy
    self.kinetic_energy = 0.
    ### potential energy
    self.potential_energy = 0
    ### mean squared velocity
    self.msv = 0.
    ### compressibility
    self.Z = 0.
    ### instantaneous temperature
    self.T_instant = 0.
    ### pressure
    self.pressure = 0. 
    ### pressure temporary quantity
    self.psum = 0.


    ### create arrays for position, force, velocity
    self.position_array = np.zeros((self.N,3))
    self.velocity_array = np.zeros((self.N,3))
    self.force_array = np.zeros((self.N,3))
    self.acceleration_array = np.zeros((self.N,3))

    self.initialize()

                          
  def initialize_velocities(self):
    ''' Initialize velocities of the particles
    based on the temperature! '''
    ### COM velocity
    v_cm = [0,0,0]
    ### initialize velocities and compute COM velocity
    for i in range(0,self.N):
      for j in range(0,3):
        self.velocity_array[i,j] = np.random.normal(0,1)
        v_cm[j] = v_cm[j] + self.velocity_array[i,j] / self.N
        
    ### now correct velocities for COM and compute squared sum of corrected velocitie
    v_sqd_sum = 0.0
    for i in range(0,self.N):
      for j in range(0,3):
        self.velocity_array[i,j] -= v_cm[j]
        v_sqd_sum = v_sqd_sum + self.velocity_array[i,j]**2
    ### scaling factor for velocities based on initial temperature
    ### and squared sum of random velocities  
    lam = np.sqrt( 3 * (self.N - 1) * self.T / v_sqd_sum)
    
    ### finally scale velocities according to initial temperature!
    for i in range(0,self.N):
      for j in range(0,3):
        self.velocity_array[i,j] *= lam
    return 1

  def initialize(self):
    ''' Initialize positions and velocities '''
    ### number of atoms in each direction
    n = int(np.ceil(self.N**(1/3.)))
    #print("n is ",n)
    ### spacing between atoms
    pos = self.L_nu / n
    #print("pos is ",pos)
    ### go through all particles and assign position
      
    p = 0
    for i in range(0,n):
      for j in range(0,n):
        for k in range(0,n):
          if p<self.N:
            self.position_array[p,0] = (i + 0.5) * pos
            self.position_array[p,1] = (j + 0.5) * pos
            self.position_array[p,2] = (k + 0.5) * pos
          p += 1
    self.initialize_velocities()
    return 1

  def compute_accelerations(self):
    ''' compute accelerations using Lennard-Jones force '''
    ### separation vector
    #r_ij = np.array([0.,0.,0.])
    ### initialize accelerations to zeros
    self.acceleration_array[:,:] = 0.0
    ''' Get list of scalar distances between particles, which will
        be used in the Force calculations, and also in total potential energy '''
    r = dist.pdist(self.position_array, 'Euclidean')
    r_a = dist.cdist(self.position_array,self.position_array,"Euclidean")
    r_a = r_a + np.identity(len(r_a))*1e12
    ''' Go ahead and compute the potential! '''
    quotient = self.sigma / r
    term1 = quotient ** 12
    term2 = quotient ** 6
    U = 4 * self.epsilon * (term1 - term2)
    self.potential_energy = np.sum(U) 
    #print("U",self.potential_energy)

    
    for i in range(0,self.N):
        c_o_i = self.position_array[i]
        r_vec = c_o_i - self.position_array
        f = 24 * (2 * r_a[i] ** (-14) - r_a[i] ** (-8))
        a_vec = np.transpose( np.multiply(f,np.transpose(r_vec)) )

        self.acceleration_array[i] = np.sum(a_vec,axis=0)

    return 1

  def velocity_verlet(self):
    ''' update velocities and positions of particles according 
    to Lennard-Jones forces '''
    ### compute accelerations from forces at current position
    self.psum = 0.
    self.compute_accelerations()

    ### update positions and do partial velocity update
    for i in range(0,self.N):
      for j in range(0,3):
        ### update positions
        self.position_array[i,j] += self.velocity_array[i,j] * self.dt 
        + 0.5 * self.acceleration_array[i,j] * self.dt ** 2
        ### update velocities
        self.velocity_array[i,j] += 0.5 * self.acceleration_array[i,j] * self.dt 

    ### now that position has been updated, update the acceleration
    self.compute_accelerations()

    ### now update the velocity with the new velocity
    for i in range(0,self.N):
      for j in range(0,3):
        self.velocity_array[i,j] += 0.5 * self.acceleration_array[i,j] * self.dt

    ### elastic walls!
    for i in range(0,self.N):
      for j in range(0,3):
        if self.position_array[i,j] < 0.:
          self.velocity_array[i,j] *= -1
          self.psum += 2 * np.abs(self.velocity_array[i,j]) / self.dt 
          
        if self.position_array[i,j] >= self.L_nu:
          self.velocity_array[i,j] *= -1
          self.psum += 2 * np.abs(self.velocity_array[i,j]) / self.dt

    ### write positions to file!
    self.pressure = self.psum / (6 * self.L_nu ** 2)

    return 1

  def mean_squared_velocity(self):
    ''' function to compute the mean squared velocity '''
    ### make sure msv is initialized to zero
    self.msv = 0.
    for i in range(0,self.N):
      for j in range(0,3):
        self.msv = self.msv + self.velocity_array[i,j] ** 2

    self.msv = self.msv / self.N
    return 1

  def kinetic(self):
    ''' function to compute the total kinetic energy '''
    self.mean_squared_velocity()
    self.kinetic_energy = self.msv * self.mass / 2
    return 1

  def potential(self):
    ''' function to compute the total potential energy '''
    ### set potential to zero
    self.potential_energy = 0.
    for i in range(0,self.N):
      for j in range(0,self.N):
        if i!=j:
          r2 = 0
          for k in range(0,3):
            r2 = r2 + (self.position_array[i,k]-self.position_array[j,k]) ** 2
          r_norm = np.sqrt(r2)
          #print("rnorm",r_norm)
          quotient = self.sigma / r_norm
          term1 = quotient ** 12.
          term2 = quotient ** 6

          self.potential_energy = self.potential_energy + 4 * self.epsilon * (term1 - term2)
    return 1


#initialize variables to store all the simulation results
atom_type_all = []
sim_temp_all = []
sim_density_all = []
Pavg_all = []
Tavg_all = []
Z_all = []

# Run MD Simulations

Now we're ready to run the MD simulations!

First select the type of gas (`atom_type`), temperature in K (`sim_temp`), and density in mol/m^3 (`sim_density`) for the simulation. Then execute the code block below to run the simulation.

(**Note:** if desired, you can also change the number of time steps in the simulation (`num_time`).)

In [3]:
### set parameters and run simulation

### set atom type (default is "Ar"; other options are "He", "Ne", "Kr", "Xe")
atom_type = "Ar"
### set simulation temperature in K (default is 300 K)
sim_temp = 300
### set density in mol/m^3 (default is 40 mol/m^3)
sim_density = 40
### set number of time steps (default is 2000)
num_time = 2000

### create noble gas instance and save output xyz file
t_gas = noble_gas(atom_type, sim_temp, sim_density)

t_gas.compute_accelerations()

Pavg = 0.
Tavg = 0.

file_name = "output_" + atom_type + "_" + str(sim_temp) + "_" + str(sim_density) +".xyz"
myfile = open(file_name,"w")
myfile.write(str(num_atoms))

for i in range(0,num_time):
  t_gas.velocity_verlet()
  ### accrue average pressure in SI
  Pavg = Pavg + t_gas.pressure * t_gas.press_fac
  t_gas.mean_squared_velocity()
  ### instantaneous temperature in natural units
  t_gas.T_instant = t_gas.msv * t_gas.mass / 3.
  ### accrue average temperature in SI
  Tavg = Tavg + t_gas.T_instant * t_gas.temp_fac
  ### np.append(all_positions,t_gas.position_array)
  for j in range(0,num_atoms):
    string="\n" + atom_type + " " + str(t_gas.position_array[j][0]) + " " + str(t_gas.position_array[j][1]) + " " + str(t_gas.position_array[j][2])
    myfile.write(string)

myfile.close()

Pavg /= num_time
Tavg /= num_time

Z = Pavg * t_gas.vol_si / (t_gas.N * t_gas.kb_SI * Tavg)

# make and display a DataFrame with the simulation results
print('Simulation Results\n')
sim_data = {'Avg. Pressure (kPa)':[Pavg/1000.],'Avg. Temp. (K):':[Tavg],'Z':[Z]}
sim_results = pd.DataFrame(data=sim_data)
sim_results = sim_results.round(decimals=3)
print(sim_results)


Simulation Results

   Avg. Pressure (kPa)  Avg. Temp. (K):      Z
0               89.773          299.669  0.901


Now let's save the simulation input parameters (atom type, initial temperature, and number density) and the output results (average pressure, average temperature, and compressibility factor).

The code block below will append the values for the latest simulation to the variables that we created previously to store the data. (**Hint:** only run this section once for each simulation or else you'll save the data twice!)

In [4]:
# add results from this simulation to the variables
atom_type_all.append(atom_type)
sim_temp_all.append(sim_temp)
sim_density_all.append(sim_density)
Pavg_all.append(Pavg/1000.)
Tavg_all.append(Tavg)
Z_all.append(Z)

#Visualize MD Simulations

Run the SECOND code block below to play a visualization of the most recent MD simulation.

If you want to visualize a different simulation result, not the most recent one, uncomment the line of code in the FIRST code block by removing the pound sign (#) and change the filename. (Hint: remember to add back the pound sign when you're done or else this code will only visualize the selected simulation.)

In [ ]:
### choose a different simulation result to visualize
### remove the leading "#" and change the simulation filename to the one you want
# file_name = 'output_Ar_300_40.xyz'

In [ ]:
### enable custom widgets (again)
from google.colab import output
output.enable_custom_widget_manager()

### create Universe object
u = MD.Universe(file_name)

print("Simulation: " + file_name)

### play MD movie
ngl.show_mdanalysis(u)

In [ ]:
### this isn't working because all_positions only has one timestep of data (maybe data type issues with all_positions)

### myfile = open("output.xyz","w")
### myfile.write(str(num_atoms))
### for i in range(0,num_time):
###   for j in range(0,num_atoms):
###     string="\n" + atom_type + str(all_positions[i][j][0]) + " " + str(all_positions[i][j][1]) + " " + str(all_positions[i][j][2])
###     myfile.write(string)
### myfile.close()


### read in XYZ coordinates -- doesn't seem necessary

### MD.coordinates.XYZ.XYZReader(file_name)

# Display Summary of Simulation Results

Now let's display a DataFrame with the results for all the simulations saved into the summary variables.

In [ ]:
### make and display a DataFrame
data_all = {'Gas':atom_type_all,'Initial Temp. (K)':sim_temp_all,'Number Density (mol/m^3)':sim_density_all,'Avg. Pressure (kPa)':Pavg_all,'Avg. Temp. (K):':Tavg_all,'Z':Z_all}
sim_results_all = pd.DataFrame(data=data_all)
sim_results_all = sim_results_all.round(decimals=3)
print(sim_results_all)

In [ ]:
### not sure what this code is doing

# from google.colab import output
# output.disable_custom_widget_manager()

#Plot the Data

Now we can analyze our simulation results further by generating different types of plots.
- A quasi-isotherm of P vs. number density
- A plot of Z vs. number density
- A plot of Z vs. T

To do this, we'll need to manually select which rows of the summary table contain the relevant data for each plot.

##Quasi-isotherms of P vs. number density

In [ ]:
# change the line below to select the rows that contain simulations done for the same gas at the same temperature
idx = [0,2]

# get x-y data and generate plot
x=sim_results_all.iloc[idx,2]
y=sim_results_all.iloc[idx,3]
plt.scatter(x, y)
plt.xlabel('Number Density (mol/m^3)')
plt.ylabel('Pressure (kPa)')
plt.show()

##Z vs. number density

In [ ]:
# change the line below to select the rows that contain simulations done for the same gas at the same temperature
idx = [0,2]

# get x-y data and generate plot
x=sim_results_all.iloc[idx,2]
y=sim_results_all.iloc[idx,5]
plt.scatter(x, y)
plt.xlabel('Number Density (mol/m^3)')
plt.ylabel('Z')
plt.show()

#Z vs. temperature

In [ ]:
# change the line below to select the rows that contain simulations done for the same gas at the same number density
idx = [0,1]

# get x-y data and generate plot
x=sim_results_all.iloc[idx,1]
y=sim_results_all.iloc[idx,5]
plt.scatter(x, y)
plt.xlabel('Temperature')
plt.ylabel('Z')
plt.show()

# Export the Data

## Summary Results

Now let's export the simulation results. We can save the summary table above as a comma separated variable (CSV) file that can be saved on your computer and opened in many different programs.

After running the code below, click on the folder icon in the left toolbar. You should see a a file called "MDsimresults.csv" in the directory. Click on the three dots to the right of the file name to download it to your computer.

In [ ]:
### save the simulation results to a CSV file called "MDsimresults.csv"
sim_results_all.to_csv('MDsimresults.csv')

## Trajectory Files

In the same directory as the summary CSV file, you should see trajectory (.xyz) files for each simulation that you ran in this session. Save those files to your computer following the same procedure for the summary CSV file. These files can be opened in another MD visualization software, such as [VMD](https://www.ks.uiuc.edu/Research/vmd/).

# Messing Around

In [ ]:
pip install mdtraj

In [ ]:
traj = mdt.load_xyz('output_Ar_300_40.xyz',top=None)

In [ ]:
import mdtraj as mdt

In [ ]:
# Read the .xyz file
with open('output_Ar_300_40.xyz') as f:
    # Skip the first line (number of atoms)
    next(f)
    # Read the rest of the lines (atom positions)
    frames = []
    for line in f:
        coords = [[float(d) for d in line.split()[1:]][i:i+3] for i in range(0, len(line.split()[1:]), 3)]
        frames.append(ngl.MDTrajTrajectory(coords))

# Create an NGLView widget
print(frames)  # Debugging print statement

view = ngl.NGLWidget(frames)

# Save each frame as a PDB file
for i, frame in enumerate(frames):
    frame.save_pdb(f'frame_{i}.pdb')
